# set global file name

In [26]:
global filename
#filename = "G1"
#N = 1000
filename = "G2"
N = 237
d = 0.85

#set iteration count as 5 for task3.2; in other situation iter_count should be 100000
iter_count = 100000
#iter_count = 4


In [27]:
#prepare nodes for algorithm
from collections import defaultdict
from collections import OrderedDict
import operator

#all the url sets
pages = set()
#all the url with no out-links
sinkset = set()
#all the url with no in-links
sources = set()
#all the edges of url graph
edges = []

#inlink dict of all url nodes
inlink = defaultdict(list)


with open(filename+".txt", "r",encoding="utf-8",newline='\n') as infile:
    for line in infile:
        nodes = line.split()
        #print(len(nodes))
        edge_dict = {}
        p = nodes[0]
        inlink[p] = []
        for i in range(1,len(nodes)):
            edge_dict[nodes[i]]=nodes[0]
            inlink[p].append(nodes[i])
        #print(edge_dict)
        #print(inlink[p])
        edges.append(edge_dict)
            
          
        for node in nodes:
            if node not in pages:
                if "," in node:
                    node = node.replace(",","")
                pages.add(node)

#print(edges)
#print("inlink:")
#print(inlink)
                
number_inlink = dict()
number_outlink = dict()
outlink = defaultdict(list)



for p in pages:
    #get page without in-link
    number_inlink[p] = len(inlink[p])
    if number_inlink[p] == 0:
        sources |= {p}
    #get out-link    
    outlink[p] = []
    for edge_set in edges:
        if p in list(edge_set.keys()):
            outlink[p].append(edge_set[p])
    number_outlink[p] = len(outlink[p])
    if number_outlink[p] == 0:
        sinkset |= {p}

#sorted nodes by in-degree        
sorted_inlink_count = OrderedDict(sorted(number_inlink.items(), key=lambda kv: kv[1],reverse=True))
#print("sorted_inlink_count")
#print(sorted_inlink_count)
save_name = filename + "sortedByinlink.txt"
f = open(save_name,"a+",encoding="utf-8",newline='\n')
for sort_link in sorted_inlink_count.items(): 
    #print(sort_link)
    message = sort_link[0] + " " + str(sort_link[1]) + '\n' 
    f.write(message)
f.close()


#sorted nodes by out-degree        
sorted_outlink_count = OrderedDict(sorted(number_outlink.items(), key=lambda kv: kv[1],reverse=True))
#print("sorted_outlink_count")
#print(sorted_outlink_count)

#print("outlink:")
#print(outlink)            
#print("number_outlink:")
#print(number_outlink)    
#print("sinkset:")
#print(sinkset)  
maximum_in =  str(next(iter(sorted_inlink_count.items()))[1])
maximum_out = str(next(iter(sorted_outlink_count.items()))[1])
print("the number of sources:")
print(str(len(sources)))
print("the number of sinks:")
print(str(len(sinkset)))
print("Maximum of in-degree:") 
print(maximum_in)    
print("Maximum of out-degree:")
print(maximum_out)

save_name = filename+"statistics.txt"
f = open(save_name,"a+",encoding="utf-8",newline='\n')
message = "the number of sources:" + str(len(sources)) + '\n' 
message += "The number of sinks:" + str(len(sinkset)) + '\n' 
message += "Maximum of in-degree:" + maximum_in + '\n'
message += "Maximum of out-degree:" + maximum_out
f.write(message)
f.close()


#set inlink(p)
#inlink[p]
#set outlink(q)
#outlink[q]    
#set number_outlink
#number_outlink[p]

the number of sources:
12
the number of sinks:
8
Maximum of in-degree:
126
Maximum of out-degree:
77


In [28]:
"""
get L1-norm value
"""
def L1_nomal(newPR,PR):
    sum_l1 = 0.0
    for page in pages:
        sum_l1 += abs(newPR[page] - PR[page])
    return sum_l1

In [29]:
import numpy as np
np.set_printoptions(suppress=True)

url_docId = dict()
with open("docIdmap.txt", "r",encoding="utf-8",newline='\n') as infile1:
    for line in infile1:
        nodes = line.split(",")
        nodes[1] = nodes[1].rstrip()
        url_docId[nodes[0]] = nodes[1]
#print(url_docId)        
PR = dict()
newPR = dict()
converge = 0;
itera = 0;

for page in pages:
    PR[page] = 1.0/N    #initial value

while True:
    sinkPR = 0
    itera += 1;
    for sink in sinkset:
        sinkPR += PR[sink]
    for p in pages:
        newPR[p] = (1-d)/N
        newPR[p] += d*sinkPR/N
        for q in inlink[p]:
            newPR[p] += d*PR[q]/number_outlink[q]
    score = L1_nomal(newPR,PR)
    save_name = filename+"by"+str(d)+"iteration.txt"
    f = open(save_name,"a+",encoding="utf-8",newline='\n')
    sorted_PR_by_value = sorted(PR.items(), key=lambda kv: kv[1],reverse=True)
    sorted_NewPR_by_value = sorted(newPR.items(), key=lambda kv: kv[1],reverse=True)
    message = "The "+str(itera)+" iterations:" +"score "+ str(score) + '\n' + str(sorted_NewPR_by_value) + '\n' + str(sorted_PR_by_value)+'\n'
    f.write(message)
    f.close()
    
    sum_pr = 0.0
    for page in pages:
        sum_pr += newPR[page]
    save_name = "L1_nomby"+str(d)+".txt"
    f = open(save_name,"a+",encoding="utf-8",newline='\n')
    message = filename + ": " + str(itera) + " iteration" + '\n' + "L1-norm: "+ str(score) + " newPR sum: " + str(sum_pr) + '\n'
    f.write(message)
    f.close()
    
    #print("The "+str(itera)+" iterations:")
    #print(newPR)
    #print(PR)
    #print(score)
    if itera > iter_count:
        print("Iterating greater than the global setting. Stop computing now! If wrong, please check the global setting")
        for pr in sorted_NewPR_by_value:
            save_name = filename+"pagerank4Timesby"+str(d)+".txt"
            f = open(save_name,"a+",encoding="utf-8",newline='\n')
            message = pr[0]+" "+str(pr[1]) + "\n"
            f.write(pr[0]+" "+str(pr[1]) + "\n")
            f.close() 
        break
    elif score > 0.001:
        for p in pages:
            PR[p] =  newPR[p]
        continue
    elif converge < 5:
        for p in pages:
            PR[p] =  newPR[p]
        converge +=1
        continue
    else:
        print("PageRank score has converged!")
        top50 = 1
        for pr in sorted_NewPR_by_value:
            save_name = filename+"pagerankby"+str(d)+".txt"
            f = open(save_name,"a+",encoding="utf-8",newline='\n')
            message = pr[0]+" "+str(pr[1]) + "\n"
            f.write(message)
            f.close() 
            if(filename == "G1"):
                try:
                    save_name = filename+"urlPageRankBy"+str(d)+".txt"
                    f = open(save_name,"a+",encoding="utf-8",newline='\n')
                    message = url_docId[pr[0]]+" "+str(pr[1]) + "\n"
                    f.write(message)
                    f.close() 
                except KeyError:
                    continue
            if(filename == "G2"):
                try:
                    save_name = filename+"urlPageRankBy"+str(d)+".txt"
                    f = open(save_name,"a+",encoding="utf-8",newline='\n')
                    message = "https://en.wikipedia.org/wiki/" + str(pr[0]) +" "+str(pr[1]) + "\n"
                    f.write(message)
                    f.close() 
                except KeyError:
                    continue
            if top50 < 51:
                save_name = filename+"pagerankTop50by"+str(d)+".txt"
                f = open(save_name,"a+",encoding="utf-8",newline='\n')
                message = pr[0]+" "+str(pr[1]) + "\n"
                f.write(pr[0]+" "+str(pr[1]) + "\n")
                f.close()
                top50 += 1
            #print(pr)
        break


PageRank score has converged!


# demo test

In [12]:
N = 6
PR = dict()
newPR = dict()
converge = 0;
d = 0.85
itera = 0;

for page in pages:
    PR[page] = 1.0/N    #initial value

while True:
    sinkPR = 0
    itera += 1;
    for sink in sinkset:
        sinkPR += PR[sink]
    for p in pages:
        newPR[p] = (1-d)/N
        newPR[p] += d*sinkPR/N
        for q in inlink[p]:
            newPR[p] += d*PR[q]/number_outlink[q]
    score = L1_nomal(newPR,PR)
    f = open("iteration.txt","a+",encoding="utf-8",newline='\n')
    sorted_PR_by_value = sorted(PR.items(), key=lambda kv: kv[1])
    sorted_NewPR_by_value = sorted(newPR.items(), key=lambda kv: kv[1])
    message = "The "+str(itera)+" iterations:" +"score "+ str(score) + '\n' + str(sorted_NewPR_by_value) + '\n' + str(sorted_PR_by_value)+'\n'
    f.write(message)
    f.close()
    #print("The "+str(itera)+" iterations:")
    #print(newPR)
    #print(PR)
    #print(score)
    if score > 0.001:
        for p in pages:
            PR[p] =  newPR[p]
        continue
    elif converge < 5:
        for p in pages:
            PR[p] =  newPR[p]
        converge +=1
        continue
    else:
        print("PageRank score has converged!")
        for pr in sorted_NewPR_by_value:
            print(pr)
        break


PageRank score has converged!
('D', 0.11890595625353888)
('B', 0.1393075828066674)
('C', 0.1513076495622817)
('F', 0.1513076495622817)
('E', 0.1870434366372427)
('A', 0.2521277251779877)


In [3]:
import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_by_value = sorted(x.items(), key=lambda kv: kv[1])
print(sorted_by_value)

[(0, 0), (2, 1), (1, 2), (4, 3), (3, 4)]


In [1]:
txt = "apple, banana, cherry"

# setting the max parameter to 1, will return a list with 2 elements!
x = txt.rsplit(", ", 1)[0]

print(x)

apple, banana
